In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from scipy.sparse import csr_matrix
import joblib

# Synthetic dataset
fake_news = [
    "Says Ohio budget item later signed into law by Gov. John Kasich requires women seeking an abortion to undergo a mandatory vaginal probe.",
    "Rep. David Cicilline is responsible for the federal loan guarantee to Solyndra, the failed solar company",
]

real_news = [
    "NEW YORK (Reuters) - A second federal judge has taken the rare step of allowing a group suing for records from Hillary Clinton’s time as U.S. secretary of state to seek sworn testimony from officials, saying there was “evidence of government wrong-doing and bad faith.” The language in Judge Royce Lamberth’s order undercut the Democratic presidential contender’s assertion she was allowed to set up a private email server in her home for her work as the country’s top diplomat and that the arrangement was not particularly unusual. He described Clinton’s email arrangement as “extraordinary” in his order filed on Tuesday in federal district court in Washington. Referring to the State Department, Clinton and Clinton’s aides, he said there had been “constantly shifting admissions by the Government and the former government officials.” Spokesmen for Clinton did not immediately respond to a request for comment.  The case is a civil matter, but the order adds to the legal uncertainty that has overshadowed Clinton’s campaign to be the Democratic nominee in the Nov. 8 presidential election. The FBI is also conducting a criminal inquiry into the arrangement after it emerged that classified government secrets ended up in Clinton’s unsecured email account. Clinton has said she does not think she will be charged with a crime.  Lamberth’s order granted the request by Judicial Watch, a conservative watchdog group suing the department under open records laws, to gather evidence, including sworn testimony. The group has filed several lawsuits, including one seeking records about the 2012 attack in Benghazi, Libya, that killed U.S. Ambassador Christopher Stevens and three other Americans. “Where there is evidence of government wrong-doing and bad faith, as here, limited discovery is appropriate, even though it is exceedingly rare in FOIA (freedom-of-information) cases,” Lamberth noted in his order. The government is normally given the benefit of the doubt that it properly searched and produced records.  Since the email arrangement came to public knowledge a year ago, the State Department has found itself defending Clinton in scores of lawsuits from groups, individuals and news outlets who say they were wrongly denied access to Clinton’s federal records. Clinton left the department in 2013, but did not return her email records to the government until nearly two years later.  Last month, Judge Emmet Sullivan, who is overseeing a separate Judicial Watch lawsuit over other Clinton-related records, allowed a similar motion for discovery. (Story refiles to fix date of presidential election, paragraph 6.)",
    "BAGHDAD (Reuters) - Iraqi forces announced on Thursday they had captured Islamic State s last stronghold in northern Iraq, leaving the militant group holed up near the Syrian border as its self-proclaimed  caliphate  shrinks further. The town of Hawija and the surrounding areas fell in an offensive by U.S.-backed Iraqi government troops and Iranian-trained and armed Shi ite paramilitary groups known as Popular Mobilisation. Some fighting took place to the north and east of the town where the militants were surrounded.  With the fall of Hawija, which lies near the Kurdish-held oil city of Kirkuk, the only area that remains under control of Islamic State in Iraq is a stretch alongside the western border with Syria, where the militant group is also in retreat. The capture of Hawija was a boost for the Iraqi government, which faces a separate crisis in the north of the country, where the Kurdish minority last month voted overwhelmingly in support of independence for their autonomous region. In Hawija, state TV showed footage of Iraqi forces raising flags in one of the town's squares while Humvees patrolled empty streets littered with car wrecks, houses riddled with bullets and shattered storefronts. (Graphics on 'Iraqi Kurds' independence push' - here)  The army s 9th armored division, the Federal Police, the Emergency Response division and ... Popular Mobilisation liberated Hawija,  said a statement from the joint operations commander, Lieutenant-General Abdul Ameer Rasheed Yarallah. Thick black smoke rose from areas surrounding Hawija, from oil wells set on fire by the militants to prevent air detection.   The capture of Hawija brings Iraqi forces into direct contact with Kurdish Peshmerga fighters who control Kirkuk, a multi-ethnic region claimed by both Baghdad and the Kurdistan Regional Government (KRG). Kirkuk shaped up as a flashpoint last month when the KRG included the city in the referendum on Kurdish independence in northern Iraq.   We don t want any aggression or confrontations but the federal authority must be imposed in the disputed areas,  Iraqi Prime Minister Haider al-Abadi told a news conference in Paris with French President Emmanuel Macron. Macron offered to mediate between the Iraqi government and Kurdish regional authorities, and promised to maintain a military presence there until Islamic State was defeated. The Iraqi government has imposed sanctions in response to the independence vote, but Abadi said he could understand the Kurds   aspirations , provided they were expressed in accordance with the constitution. The outcome of the referendum has raised concerns in Iraq and abroad that a conflict might break out there along ethnic lines, potentially weakening the campaign against Islamic State. Turkey, which is battling a three-decade-old insurgency by its own Kurdish militants, has led regional opposition to the Iraqi Kurds  independence hopes. Stepping up his rhetoric on the issue, President Tayyip Erdogan said Turkey would soon close its border with northern Iraq and shut its air space in response to the referendum. He added that Turkey, Iran and Iraq would jointly decide on closing the flow of oil from northern Iraq, a vital source of earnings for Iraqi Kurdistan. On Wednesday, Russian President Vladimir Putin, whose country has big energy interests in Kurdistan, said it was in no one s interest to cut off oil supplies from the territory.  But Erdogan said that  if a decision will be made on closing oil taps in the region, that will be made by us. Turkey, Iran and Iraq s central government will do so together . Erdogan s stance has been backed by Iran. After meeting Erdogan in Tehran on Wednesday, Supreme Leader Ayatollah Ali Khamenei said Iran and Turkey should prevent the Iraqi Kurds from declaring independence. Relations have been cool between Shi ite Iran and mainly Sunni Turkey. But both have been alarmed by the independence vote, fearing it will encourage separatism among their own Kurdish populations. The Kurds are the region s fourth-largest ethnic group, spread across Iran, Turkey, Syria and Iraq, all of which oppose any moves towards a Kurdish state. Iraq launched an offensive on Sept. 21 to dislodge Islamic State from the Hawija area, where up to 78,000 people were estimated to be trapped, according to the United Nations.  The militants control the border town of al-Qaim and the region surrounding it.  They also hold parts of the Syrian side of the border, but the area under their control is shrinking as they retreat in the face of two different sets of hostile forces - a U.S.-backed, Kurdish-led coalition and Syrian government troops with foreign Shi ite militias backed by Iran and Russia. Islamic State s cross-border  caliphate  effectively collapsed in July, when U.S.-backed Iraqi forces captured Mosul, the group s de facto capital in Iraq, in a grueling battle which lasted nine months. The militants  leader, Abu Bakr al-Baghdadi, who declared the caliphate from Mosul in mid-2014, released an audio recording last week that indicated he was alive, after several reports he had been killed. He urged his followers to keep up the fight despite setbacks."
]

# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercase
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Remove stopwords and lemmatize
    return ' '.join(filtered_tokens)

# Creating a synthetic dataset
data = pd.DataFrame({'text': fake_news + real_news, 'label': [1]*len(fake_news) + [0]*len(real_news)})

# Load the TF-IDF vectorizer
tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')

# Load the TruncatedSVD model
svd = joblib.load('truncated_svd_model_0.8.joblib')

# Load the SVM Model
model = joblib.load('svm_model_tfidf_9676.joblib')

# Preprocess the synthetic dataset
text_test_preprocessed = data['text'].apply(preprocess_text)

# Transform the synthetic dataset using pre-trained models
test_tfidf_matrix = tfidf_vectorizer.transform(text_test_preprocessed)
dense_test_tfidf_matrix = svd.transform(test_tfidf_matrix)
test_labels = data['label'].values

# Predict and evaluate
test_predictions = model.predict(dense_test_tfidf_matrix)
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f'Test accuracy: {test_accuracy}')

# Generate classification report
report = classification_report(test_labels, test_predictions)
print(test_predictions)
print(report)

Test accuracy: 1.0
[1 1 0 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

